In [1]:
import numpy as np
import tensorflow as tf
from mtcnn import MTCNN
from numpy import asarray
from PIL import Image

from keras_vggface.utils import preprocess_input
from PIL import Image
import io
import threading
from flask import Flask, request, jsonify
import numpy as np
import pandas as pd
import io
import threading

In [2]:
csv_file = 'results.csv'
data = pd.read_csv(csv_file)


Extract Face

In [3]:
vgg_face_64 = tf.keras.models.load_model('vgg_face_model_64.h5')

app = Flask(__name__)

def extract_face(pixels, required_size=(224, 224)):

    detector = MTCNN()
	
    results = detector.detect_faces(pixels)
    x1, y1, width, height = results[0]['box']
    x2, y2 = x1 + width, y1 + height
	
    face = pixels[y1:y2, x1:x2]
	
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    face_array = face_array.astype('float32')
    
    img_array = np.expand_dims(face_array, axis=0)
    img_array = preprocess_input(img_array, version=2)
    
    return img_array


def generate_hashcode(img):
    img_array = extract_face(img)
    vect = vgg_face_64.predict(img_array)
    hashcode = np.where(vect >= 0.5, 1, 0)
    return hashcode


# Function to compare vectors and show information if the difference is less than the threshold
def compare_and_get_info(image, threshold=5):
    
    hashcode_64 = generate_hashcode(image)
    
    for index, row in (data.iterrows()):
        csv_vector = row[1]
        vector_str_clean = csv_vector.replace("\n", "").replace("[", "").replace("]", "").strip()
        csv_vector = np.array(list(map(int, vector_str_clean.split())), dtype=int)
        
        differences = np.sum(hashcode_64 != csv_vector) 
        
        if differences < threshold:
            matched_person = {
            'Image' : row[0],
            'Name': row[2],
            'Age': row[3],
            'Job': row[4]
            }
            return matched_person
    return None
            

In [4]:
@app.route('/get_infos', methods=['POST'])
def get_infos():
	if 'image' not in request.files:
		return jsonify({'error': 'No image provided'}), 400

	file = request.files['image']
	if file.filename == '':
		return jsonify({'error': 'No selected file'}), 400

	img = Image.open(io.BytesIO(file.read()))
	img = np.array(img)

	matched_person = compare_and_get_info(img)
	if matched_person:
		return jsonify(matched_person), 200
	else:
		return jsonify({'error': 'No match found'}), 404

In [ ]:
def run_app():
    app.run(debug=False, use_reloader=False)

# Run Flask app in a separate thread
thread = threading.Thread(target=run_app)
thread.start()

 * Serving Flask app '__main__'
 * Debug mode: off



 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


1/1 [==============================] - 2s 2s/step


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_35940\2751737504.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  csv_vector = row[1]
127.0.0.1 - - [09/Jan/2025 17:23:15] "POST /get_infos HTTP/1.1" 404 -


1/1 [==============================] - 0s 123ms/step


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_35940\2751737504.py:47: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'Image' : row[0],
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_35940\2751737504.py:48: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'Name': row[2],
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_35940\2751737504.py:49: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'Age': row[3],
C:\Users\LENOVO\AppData\Local\Temp\ipykern

1/1 [==============================] - 0s 133ms/step


127.0.0.1 - - [09/Jan/2025 17:29:44] "POST /get_infos HTTP/1.1" 200 -


1/1 [==============================] - 0s 114ms/step


127.0.0.1 - - [09/Jan/2025 17:32:09] "POST /get_infos HTTP/1.1" 404 -


In [31]:
import requests

url = 'http://127.0.0.1:5000/get_infos'
image_path = r"C:\Users\LENOVO\Pictures\WIN_20230220_17_14_34_Pro.jpg"
with open(image_path, 'rb') as img_file:
    files = {'image': img_file}
    response = requests.post(url, files=files)

if response.status_code == 200:
    print("Age:", response.json().get('Age'))
else:
    print("Error:", response.json().get('error', 'Unknown error'))


Age: 22
